In [2]:
import sys
sys.path.append('/home/airg/rabedi/UCSB')

In [3]:
import os
import pickle
import pandas as pd
import rasterio
from datatorch import *
from tqdm import tqdm

In [ ]:
# Configuration
usage = "train"  # "train", "validate",
pickle_dir = "./pickles/finetune"
pickle_file = os.path.join(pickle_dir, f"{usage}_data.pkl")
catalog_csv = "./finetune.csv"
data_path = "/home/airg/data/labels/cropland"

os.makedirs(pickle_dir, exist_ok=True)

catalog_df = pd.read_csv(catalog_csv)

# initializing dataset
dataset = ImageData(
    data_path=data_path,
    log_dir="./logs",
    catalog=catalog_df,
    data_size=224,
    buffer=0,
    buffer_comp=0,
    usage=usage, 
    img_path_cols="image",
    label_path_col="label",
    label_group=[0, 1, 2, 3, 4],
    apply_normalization=True,
    normal_strategy="min_max",
    stat_procedure="lab",
    global_stats=None,
    trans=None,
    parallel=False,
    clip_val=0
)

metadata_list = []
for rel_path in dataset.catalog["image"]:
    full_path = os.path.join(data_path, rel_path)
    try:
        with rasterio.open(full_path) as src:
            metadata_list.append(src.meta)
    except Exception as e:
        print(f"Warning: Failed to read metadata from {full_path}: {e}")
        metadata_list.append(None)

to_save = {
    "images": dataset.img,                                      # list of image arrays
    "labels": dataset.label,                                    # list of label arrays
    "image_paths": dataset.catalog["image"].tolist(),           # relative image file paths
    "label_paths": dataset.catalog["label"].tolist(),           # relative label file paths
    "metadata": metadata_list                                   # rasterio metadata for each image
}

with open(pickle_file, "wb") as f:
    pickle.dump(to_save, f)

print(f"Saved {len(dataset.img)} samples to {pickle_file}")

In [7]:
import os
import pickle
import pandas as pd
from datatorch import *

# === Configuration ===
catalog_csv = "./pretrain.csv"
data_path = "/home/airg/data/labels/cropland"
pickle_dir = "./pickles/pretrain"
pickle_file = os.path.join(pickle_dir, "test_data.pkl")

os.makedirs(pickle_dir, exist_ok=True)

# === Load catalog and find test rows ===
catalog_df = pd.read_csv(catalog_csv)
test_indices = catalog_df[catalog_df["usage"] == "test"].index

# === Initialize containers ===
all_chips = []
all_chip_indices = []
all_image_paths = []
all_metadata = []
all_tile_ids = []
all_years = []

# === Loop through each test tile ===
for catalog_index in tqdm(test_indices, desc="Processing test tiles"):
    dataset = ImageData(
        data_path=data_path,
        log_dir="./logs",
        catalog=catalog_df.copy(),
        data_size=224,
        buffer=0,              
        buffer_comp=0,
        usage="predict",
        img_path_cols="image",
        catalog_index=catalog_index,
        apply_normalization=True,
        normal_strategy="min_max",
        stat_procedure="lab",
        global_stats=None,
        trans=None,
        parallel=False,
        clip_val=0
    )

    # Append each tile’s chips and metadata
    all_chips.extend(dataset.img)
    all_chip_indices.extend(dataset.index)
    all_metadata.extend([dataset.meta] * len(dataset.img))
    all_image_paths.extend([dataset.catalog[dataset.img_cols[0]]] * len(dataset.img))
    all_tile_ids.extend([dataset.tile] * len(dataset.img))
    all_years.extend([dataset.year] * len(dataset.img))

# === Final pickle object ===
to_save = {
    "chips": all_chips,
    "chip_indices": all_chip_indices,
    "metadata": all_metadata,
    "image_paths": all_image_paths,
    "tile_ids": all_tile_ids,
    "years": all_years
}

# === Save to file ===
with open(pickle_file, "wb") as f:
    pickle.dump(to_save, f)

print(f"All test tiles saved to {pickle_file} with {len(all_chips)} total chips")


Processing test tiles:   0%|                   | 8/3779 [00:00<00:51, 73.64it/s]

----------Prediction data loaded for tile BF0431785, year 2017-----------
----------Prediction data loaded for tile SD3918874, year 2018-----------
----------Prediction data loaded for tile SN0393402, year 2020-----------
----------Prediction data loaded for tile ML3544736, year 2019-----------
----------Prediction data loaded for tile SD4628707, year 2018-----------
----------Prediction data loaded for tile SN0572586, year 2017-----------
----------Prediction data loaded for tile SD4738223, year 2019-----------
----------Prediction data loaded for tile NG1224862, year 2019-----------
----------Prediction data loaded for tile CM0100290, year 2017-----------
----------Prediction data loaded for tile ET1481065, year 2022-----------
----------Prediction data loaded for tile ET2310040, year 2021-----------
----------Prediction data loaded for tile ET2366964, year 2020-----------
----------Prediction data loaded for tile UG0378067, year 2022-----------
----------Prediction data loaded for t

Processing test tiles:   1%|                  | 24/3779 [00:00<00:51, 72.28it/s]

----------Prediction data loaded for tile CM0246406, year 2018-----------
----------Prediction data loaded for tile KE1648498, year 2021-----------
----------Prediction data loaded for tile KE0870366, year 2021-----------
----------Prediction data loaded for tile UG0307480, year 2021-----------
----------Prediction data loaded for tile KE1318589, year 2020-----------
----------Prediction data loaded for tile CD5627976, year 2018-----------
----------Prediction data loaded for tile CD6788719, year 2022-----------
----------Prediction data loaded for tile SO1566607, year 2020-----------
----------Prediction data loaded for tile NE3404049, year 2018-----------
----------Prediction data loaded for tile ET0071185, year 2018-----------
----------Prediction data loaded for tile SN0408719, year 2017-----------
----------Prediction data loaded for tile BF0531383, year 2021-----------
----------Prediction data loaded for tile BF0878324, year 2018-----------
----------Prediction data loaded for t

Processing test tiles:   1%|▏                 | 40/3779 [00:00<00:51, 72.86it/s]

...

Processing test tiles: 100%|███████████████▉| 3771/3779 [00:51<00:00, 71.36it/s]

----------Prediction data loaded for tile SD4666090, year 2019-----------
----------Prediction data loaded for tile NG0761816, year 2019-----------
----------Prediction data loaded for tile TZ0714909, year 2017-----------
----------Prediction data loaded for tile ET0129793, year 2022-----------
----------Prediction data loaded for tile NG1602052, year 2023-----------


Processing test tiles: 100%|████████████████| 3779/3779 [00:51<00:00, 73.11it/s]

----------Prediction data loaded for tile MR3564087, year 2020-----------
----------Prediction data loaded for tile ET0883111, year 2018-----------
----------Prediction data loaded for tile NE3795474, year 2022-----------
----------Prediction data loaded for tile TD4254042, year 2017-----------


All test tiles saved to ./pickles/pretrain/test_data.pkl with 3779 total chips


In [47]:
import os
import pickle
import pandas as pd
from pathlib import Path
from datatorch import *
from tqdm import tqdm

# === Configuration ===
catalog_csv = "./finetune.csv"
data_path = "/home/airg/data/labels/cropland"
pickle_dir = "./pickles/finetune"
output_file = os.path.join(pickle_dir, "test_data_lbl.pkl")

os.makedirs(pickle_dir, exist_ok=True)

# === Load catalog and find test rows ===
catalog_df = pd.read_csv(catalog_csv)
test_indices = catalog_df[catalog_df["usage"] == "test"].index

# === Initialize containers ===
all_chips = []
all_chip_indices = []
all_lbl_paths = []
all_metadata = []
all_tile_ids = []
all_years = []

# === Loop through each test tile ===
for catalog_index in tqdm(test_indices, desc="Processing test tiles"):
    dataset = ImageData(
        data_path=data_path,
        log_dir="./work_dir/logs",
        catalog=catalog_df.copy(),
        data_size=224,
        buffer=0,              
        buffer_comp=0,
        usage="predict",
        img_path_cols="label",
        catalog_index=catalog_index,
        apply_normalization=False,
        normal_strategy="min_max",
        stat_procedure="lab",
        global_stats=None,
        trans=None,
        parallel=False,
        clip_val=0
    )

    # Add image chips and associated metadata
    all_chips.extend(dataset.img)
    all_chip_indices.extend(dataset.index)
    all_metadata.extend([dataset.meta] * len(dataset.img))
    all_tile_ids.extend([dataset.tile] * len(dataset.img))
    all_years.extend([dataset.year] * len(dataset.img))

    full_path = dataset.catalog["label"]
    filename = Path(full_path).name
    parts = filename.replace(".tif", "").split("_")

    # Format as GHxxxxx_yyyy-mm.tif
    if len(parts) == 3:
        new_filename = f"{parts[0]}_{parts[2]}.tif"
    else:
        new_filename = filename

    print(new_filename, filename)
    all_lbl_paths.extend([new_filename] * len(dataset.img))

# === Final pickle object ===
to_save = {
    "chips": all_chips,
    "chip_indices": all_chip_indices,
    "metadata": all_metadata,
    "label_paths": all_lbl_paths,
    "tile_ids": all_tile_ids,
    "years": all_years
}

# === Save to file ===
with open(output_file, "wb") as f:
    pickle.dump(to_save, f)

print(f"All test tiles saved to {output_file} with {len(all_chips)} total chips")


Processing test tiles:  13%|█████████████████████████▏                                                                                                                                                                    | 20/151 [00:00<00:01, 103.21it/s]

----------Prediction data loaded for tile GH0030800, year 2017-----------
GH0030800_2017-02.tif GH0030800_4172_2017-02.tif
----------Prediction data loaded for tile GH0041214, year 2020-----------
GH0041214_2020-02.tif GH0041214_5300_2020-02.tif
----------Prediction data loaded for tile GH0457305, year 2019-----------
GH0457305_2019-02.tif GH0457305_6388_2019-02.tif
----------Prediction data loaded for tile GH0395862, year 2019-----------
GH0395862_2019-02.tif GH0395862_7421_2019-02.tif
----------Prediction data loaded for tile GH0662635, year 2018-----------
GH0662635_2018-02.tif GH0662635_8508_2018-02.tif
----------Prediction data loaded for tile GH0218520, year 2021-----------
GH0218520_2021-04.tif GH0218520_9589_2021-04.tif
----------Prediction data loaded for tile GH0344048, year 2018-----------
GH0344048_2018-02.tif GH0344048_9595_2018-02.tif
----------Prediction data loaded for tile GH0187827, year 2022-----------
GH0187827_2022-03.tif GH0187827_13974_2022-03.tif
----------Predi

Processing test tiles:  38%|████████████████████████████████████████████████████████████████████████▉                                                                                                                     | 58/151 [00:00<00:00, 164.93it/s]

----------Prediction data loaded for tile GH0056928, year 2023-----------
GH0056928_2023-04.tif GH0056928_33717_2023-04.tif
----------Prediction data loaded for tile GH0573798, year 2019-----------
GH0573798_2019-02.tif GH0573798_34188_2019-02.tif
----------Prediction data loaded for tile GH0757903, year 2023-----------
GH0757903_2023-12.tif GH0757903_34701_2023-12.tif
----------Prediction data loaded for tile GH0423394, year 2023-----------
GH0423394_2023-12.tif GH0423394_37219_2023-12.tif
----------Prediction data loaded for tile GH0286576, year 2018-----------
GH0286576_2018-12.tif GH0286576_2018-12.tif
----------Prediction data loaded for tile GH0124483, year 2018-----------
GH0124483_2018-12.tif GH0124483_2018-12.tif
----------Prediction data loaded for tile GH0282000, year 2018-----------
GH0282000_2018-12.tif GH0282000_2018-12.tif
----------Prediction data loaded for tile GH0164971, year 2018-----------
GH0164971_2018-12.tif GH0164971_2018-12.tif
----------Prediction data loaded

Processing test tiles:  69%|██████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████▏                                                          | 104/151 [00:00<00:00, 199.66it/s]

----------Prediction data loaded for tile GH0077402, year 2018-----------
GH0077402_2018-12.tif GH0077402_2018-12.tif
----------Prediction data loaded for tile GH0039037, year 2018-----------
GH0039037_2018-12.tif GH0039037_2018-12.tif
----------Prediction data loaded for tile GH0588269, year 2018-----------
GH0588269_2018-12.tif GH0588269_2018-12.tif
----------Prediction data loaded for tile GH0664214, year 2018-----------
GH0664214_2018-12.tif GH0664214_2018-12.tif
----------Prediction data loaded for tile GH0588433, year 2018-----------
GH0588433_2018-12.tif GH0588433_2018-12.tif
----------Prediction data loaded for tile GH0002362, year 2018-----------
GH0002362_2018-12.tif GH0002362_2018-12.tif
----------Prediction data loaded for tile GH0473216, year 2020-----------
GH0473216_2020-11.tif GH0473216_2020-11.tif
----------Prediction data loaded for tile GH0638914, year 2018-----------
GH0638914_2018-12.tif GH0638914_2018-12.tif
----------Prediction data loaded for tile GH0474733, yea

Processing test tiles:  82%|███████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████▏                                 | 124/151 [00:00<00:00, 169.85it/s]

----------Prediction data loaded for tile GH0433491, year 2022-----------
GH0433491_2022-12.tif GH0433491_22321_2022-12.tif
----------Prediction data loaded for tile GH0251385, year 2018-----------
GH0251385_2018-02.tif GH0251385_13347_2018-02.tif
----------Prediction data loaded for tile GH0682438, year 2019-----------
GH0682438_2019-02.tif GH0682438_22593_2019-02.tif
----------Prediction data loaded for tile GH0288179, year 2020-----------
GH0288179_2020-12.tif GH0288179_19121_2020-12.tif
----------Prediction data loaded for tile GH0176860, year 2018-----------
GH0176860_2018-02.tif GH0176860_41113_2018-02.tif
----------Prediction data loaded for tile GH0163936, year 2017-----------
GH0163936_2017-02.tif GH0163936_630_2017-02.tif
----------Prediction data loaded for tile GH0624670, year 2023-----------
GH0624670_2023-03.tif GH0624670_30536_2023-03.tif
----------Prediction data loaded for tile GH0644788, year 2017-----------
GH0644788_2017-02.tif GH0644788_22390_2017-02.tif
----------

Processing test tiles: 100%|█████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████| 151/151 [00:00<00:00, 153.33it/s]

GH0747247_2020-12.tif GH0747247_9600_2020-12.tif
----------Prediction data loaded for tile GH0141970, year 2020-----------
GH0141970_2020-02.tif GH0141970_13970_2020-02.tif
----------Prediction data loaded for tile GH0285763, year 2022-----------
GH0285763_2022-12.tif GH0285763_27134_2022-12.tif
----------Prediction data loaded for tile GH0090018, year 2017-----------
GH0090018_2017-08.tif GH0090018_13969_2017-08.tif
----------Prediction data loaded for tile GH0097202, year 2021-----------
GH0097202_2021-04.tif GH0097202_22508_2021-04.tif
----------Prediction data loaded for tile GH0064654, year 2020-----------
GH0064654_2020-02.tif GH0064654_6374_2020-02.tif
----------Prediction data loaded for tile GH0345353, year 2019-----------
GH0345353_2019-02.tif GH0345353_34693_2019-02.tif
----------Prediction data loaded for tile GH0065558, year 2020-----------
GH0065558_2020-02.tif GH0065558_22499_2020-02.tif
----------Prediction data loaded for tile GH0635606, year 2019-----------
GH0635606_

In [48]:
import pickle

# Path to your pickle file
pickle_file = "./pickles/finetune/test_data_lbl.pkl"

# Load the pickle
with open(pickle_file, "rb") as f:
    data = pickle.load(f)

# Print the first 10 label file names
print("Sample label file names:")
for i, path in enumerate(data["label_paths"][:150]):
    print(f"{i+1}: {path}")


Sample label file names:
1: GH0030800_2017-02.tif
2: GH0041214_2020-02.tif
3: GH0457305_2019-02.tif
4: GH0395862_2019-02.tif
5: GH0662635_2018-02.tif
6: GH0218520_2021-04.tif
7: GH0344048_2018-02.tif
8: GH0187827_2022-03.tif
9: GH0245065_2021-04.tif
10: GH0307766_2022-12.tif
11: GH0012756_2017-02.tif
12: GH0086254_2018-08.tif
13: GH0199239_2018-02.tif
14: GH0224089_2020-12.tif
15: GH0218490_2019-02.tif
16: GH0236273_2022-12.tif
17: GH0232753_2021-04.tif
18: GH0552545_2021-03.tif
19: GH0189383_2022-04.tif
20: GH0300581_2020-02.tif
21: GH0171128_2017-02.tif
22: GH0123422_2023-08.tif
23: GH0525092_2023-12.tif
24: GH0215837_2018-02.tif
25: GH0240814_2022-12.tif
26: GH0056928_2023-04.tif
27: GH0573798_2019-02.tif
28: GH0757903_2023-12.tif
29: GH0423394_2023-12.tif
30: GH0286576_2018-12.tif
31: GH0124483_2018-12.tif
32: GH0282000_2018-12.tif
33: GH0164971_2018-12.tif
34: GH0372623_2020-11.tif
35: GH0299784_2020-11.tif
36: GH0087912_2018-12.tif
37: GH0117889_2018-12.tif
38: GH0196342_2018-12.

## Pickling 2 classes

In [51]:
import os
import pickle
import pandas as pd
import numpy as np
from pathlib import Path
from datatorch import ImageData
from tqdm import tqdm

# === Configuration ===
catalog_csv = "./finetune.csv"
data_path = "/home/airg/data/labels/cropland"
pickle_dir = "./pickles/finetune"
output_file = os.path.join(pickle_dir, "test_data_lbl_2cls.pkl")

os.makedirs(pickle_dir, exist_ok=True)

# === Load catalog and find test rows ===
catalog_df = pd.read_csv(catalog_csv)
test_indices = catalog_df[catalog_df["usage"] == "test"].index

# === Initialize containers ===
all_chips = []
all_chip_indices = []
all_lbl_paths = []
all_metadata = []
all_tile_ids = []
all_years = []

# === Loop through each test tile ===
for catalog_index in tqdm(test_indices, desc="Processing test tiles"):
    dataset = ImageData(
        data_path=data_path,
        log_dir="./work_dir/logs",
        catalog=catalog_df.copy(),
        data_size=224,
        buffer=0,
        buffer_comp=0,
        usage="predict",
        img_path_cols="label",
        catalog_index=catalog_index,
        apply_normalization=False,
        normal_strategy="min_max",
        stat_procedure="lab",
        global_stats=None,
        trans=None,
        parallel=False,
        clip_val=0
    )

    # Convert labels: replace 2 with 1 to create binary labels
    binary_labels = [np.where(chip == 2, 1, chip) for chip in dataset.img]

    all_chips.extend(binary_labels)
    all_chip_indices.extend(dataset.index)
    all_metadata.extend([dataset.meta] * len(binary_labels))
    all_tile_ids.extend([dataset.tile] * len(binary_labels))
    all_years.extend([dataset.year] * len(binary_labels))

    full_path = dataset.catalog["label"]
    filename = Path(full_path).name
    parts = filename.replace(".tif", "").split("_")

    # Format as GHxxxxx_yyyy-mm.tif
    if len(parts) == 3:
        new_filename = f"{parts[0]}_{parts[2]}.tif"
    else:
        new_filename = filename

    print(new_filename, filename)
    all_lbl_paths.extend([new_filename] * len(binary_labels))

# === Final pickle object ===
to_save = {
    "chips": all_chips,
    "chip_indices": all_chip_indices,
    "metadata": all_metadata,
    "label_paths": all_lbl_paths,
    "tile_ids": all_tile_ids,
    "years": all_years
}

# === Save to file ===
with open(output_file, "wb") as f:
    pickle.dump(to_save, f)

print(f"All test tiles saved to {output_file} with {len(all_chips)} total chips")


Processing test tiles:   7%|████████████▋                                                                                                                                                                                  | 10/151 [00:00<00:06, 23.10it/s]

----------Prediction data loaded for tile GH0030800, year 2017-----------
GH0030800_2017-02.tif GH0030800_4172_2017-02.tif
----------Prediction data loaded for tile GH0041214, year 2020-----------
GH0041214_2020-02.tif GH0041214_5300_2020-02.tif
----------Prediction data loaded for tile GH0457305, year 2019-----------
GH0457305_2019-02.tif GH0457305_6388_2019-02.tif
----------Prediction data loaded for tile GH0395862, year 2019-----------
GH0395862_2019-02.tif GH0395862_7421_2019-02.tif
----------Prediction data loaded for tile GH0662635, year 2018-----------
GH0662635_2018-02.tif GH0662635_8508_2018-02.tif
----------Prediction data loaded for tile GH0218520, year 2021-----------
GH0218520_2021-04.tif GH0218520_9589_2021-04.tif
----------Prediction data loaded for tile GH0344048, year 2018-----------
GH0344048_2018-02.tif GH0344048_9595_2018-02.tif
----------Prediction data loaded for tile GH0187827, year 2022-----------
GH0187827_2022-03.tif GH0187827_13974_2022-03.tif
----------Predi

Processing test tiles:  23%|████████████████████████████████████████████▎                                                                                                                                                  | 35/151 [00:00<00:01, 67.49it/s]

----------Prediction data loaded for tile GH0300581, year 2020-----------
GH0300581_2020-02.tif GH0300581_28615_2020-02.tif
----------Prediction data loaded for tile GH0171128, year 2017-----------
GH0171128_2017-02.tif GH0171128_29745_2017-02.tif
----------Prediction data loaded for tile GH0123422, year 2023-----------
GH0123422_2023-08.tif GH0123422_31030_2023-08.tif
----------Prediction data loaded for tile GH0525092, year 2023-----------
GH0525092_2023-12.tif GH0525092_32218_2023-12.tif
----------Prediction data loaded for tile GH0215837, year 2018-----------
GH0215837_2018-02.tif GH0215837_32323_2018-02.tif
----------Prediction data loaded for tile GH0240814, year 2022-----------
GH0240814_2022-12.tif GH0240814_32595_2022-12.tif
----------Prediction data loaded for tile GH0056928, year 2023-----------
GH0056928_2023-04.tif GH0056928_33717_2023-04.tif
----------Prediction data loaded for tile GH0573798, year 2019-----------
GH0573798_2019-02.tif GH0573798_34188_2019-02.tif
--------

Processing test tiles:  30%|████████████████████████████████████████████████████████▉                                                                                                                                      | 45/151 [00:00<00:01, 62.94it/s]

...

Processing test tiles:  91%|████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████▍                 | 137/151 [00:02<00:00, 44.39it/s]

----------Prediction data loaded for tile GH0056531, year 2018-----------
GH0056531_2018-02.tif GH0056531_22498_2018-02.tif
----------Prediction data loaded for tile GH0570911, year 2017-----------
GH0570911_2017-02.tif GH0570911_33031_2017-02.tif
----------Prediction data loaded for tile GH0540994, year 2018-----------
GH0540994_2018-08.tif GH0540994_22584_2018-08.tif
----------Prediction data loaded for tile GH0747247, year 2020-----------
GH0747247_2020-12.tif GH0747247_9600_2020-12.tif
----------Prediction data loaded for tile GH0141970, year 2020-----------
GH0141970_2020-02.tif GH0141970_13970_2020-02.tif
----------Prediction data loaded for tile GH0285763, year 2022-----------
GH0285763_2022-12.tif GH0285763_27134_2022-12.tif


Processing test tiles: 100%|██████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████| 151/151 [00:02<00:00, 52.14it/s]

----------Prediction data loaded for tile GH0090018, year 2017-----------
GH0090018_2017-08.tif GH0090018_13969_2017-08.tif
----------Prediction data loaded for tile GH0097202, year 2021-----------
GH0097202_2021-04.tif GH0097202_22508_2021-04.tif
----------Prediction data loaded for tile GH0064654, year 2020-----------
GH0064654_2020-02.tif GH0064654_6374_2020-02.tif
----------Prediction data loaded for tile GH0345353, year 2019-----------
GH0345353_2019-02.tif GH0345353_34693_2019-02.tif
----------Prediction data loaded for tile GH0065558, year 2020-----------
GH0065558_2020-02.tif GH0065558_22499_2020-02.tif
----------Prediction data loaded for tile GH0635606, year 2019-----------
GH0635606_2019-02.tif GH0635606_37667_2019-02.tif
----------Prediction data loaded for tile GH0266377, year 2019-----------
GH0266377_2019-02.tif GH0266377_31279_2019-02.tif
----------Prediction data loaded for tile GH0235985, year 2018-----------
GH0235985_2018-02.tif GH0235985_9591_2018-02.tif
----------

All test tiles saved to ./pickles/finetune/test_data_lbl_2cls.pkl with 151 total chips


In [44]:
new_filename, filename

('GH0304027_2020-12.tif', 'GH0304027_1914_2020-12.tif')